<a href="https://colab.research.google.com/github/diegosimao/An-lise-Airbnb/blob/main/CASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno 
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif
from datetime import datetime
import xgboost as xgb

from sklearn.model_selection import train_test_split
import seaborn as sns

#Lendo arquvio
conn = sqlite3.connect('/content/case_ds_gdem.sqlite3')
#Transformando sqlite em um dataframe
df = pd.read_sql_query("SELECT * FROM vendas", conn)


In [ ]:

display(df.dtypes)
df.count

In [ ]:
df.head()

In [ ]:
#df.COD_CICLO = df.COD_CICLO.astype(str)
df['COD_CICLO'] = pd.to_datetime(df['COD_CICLO'], format='%Y%m',exact=False)
#df['COD_CICLO'] = pd.to_datetime(df['COD_CICLO'],format='%Y%m')
df.head()

In [ ]:
#Estatística básica dos dados

df[['VL_PRECO', 'QT_VENDA']].describe()



In [ ]:
df.COD_CICLO.min(),df.COD_CICLO.max()

In [ ]:
#df['COD_CICLO'] = pd.to_datetime(df['COD_CICLO']).dt.strftime('%d-%m-%Y')

In [ ]:
plt.hist(df.QT_VENDA, bins=5, rwidth=0.8)
plt.xlabel('Height (inches)')
plt.ylabel('Count')
plt.show()

In [ ]:
(df.isnull().sum() / df.shape[0]).sort_values(ascending=False)

In [ ]:
#Visualização gráfica da qualidade dos dados
msno.bar (df, figsize = (8,3));

In [ ]:
df.hist(bins=15, figsize=(17,17));

In [ ]:
limite_inferior = df.QT_VENDA.mean() + 3*df.QT_VENDA.std()
limite_inferior

In [ ]:
limite_superior = df.QT_VENDA.mean() -3*df.QT_VENDA.std()
limite_superior

In [ ]:
outlier = df[(df.QT_VENDA>limite_inferior) | (df.QT_VENDA<limite_superior)]


In [ ]:
outlier.count

In [ ]:
df_sem_outlier = df[(df.QT_VENDA<limite_inferior) & (df.QT_VENDA>limite_superior)]
df_sem_outlier.head()

In [ ]:
df_sem_outlier.shape

In [ ]:

df.shape

In [ ]:
plt.hist(df_sem_outlier.QT_VENDA, bins=10, rwidth=0.8)
plt.xlabel('Height (inches)')
plt.ylabel('-')
plt.show()

In [ ]:
label_encoder = LabelEncoder()
labels_DES_CATEGORIA_MATERIAL = label_encoder.fit_transform(df_sem_outlier.DES_CATEGORIA_MATERIAL)  
labels_DES_CATEGORIA_MATERIAL

In [ ]:
label_encoder = LabelEncoder()
labels_DES_MARCA_MATERIAL = label_encoder.fit_transform(df_sem_outlier.DES_MARCA_MATERIAL         )  
labels_DES_MARCA_MATERIAL

In [ ]:
df_sem_outlier['Marca_Material'] = labels_DES_MARCA_MATERIAL
df_sem_outlier['Categoria_Material'] = labels_DES_CATEGORIA_MATERIAL
del df_sem_outlier['DES_CATEGORIA_MATERIAL']
del df_sem_outlier['DES_MARCA_MATERIAL']

df_sem_outlier['PCT_DESCONTO'] = df['PCT_DESCONTO'].fillna(0)

In [ ]:
df_sem_outlier = df_sem_outlier.sort_values("COD_CICLO")

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(df_sem_outlier['COD_CICLO'], df_sem_outlier['QT_VENDA'])


In [ ]:
X = df_sem_outlier.drop(['QT_VENDA','COD_CICLO'], axis=1)
y = df_sem_outlier['QT_VENDA']


In [ ]:
X.isnull().sum()

In [ ]:
missing_col = ['VL_PRECO']
for i in missing_col:
 X.loc[X.loc[:,i].isnull(),i]=X.loc[:,i].mean()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)# Treinando modelo


In [ ]:
model = RandomForestClassifier(n_estimators=10,max_depth=6)
model.fit(X_train, y_train)# Mostrando importância de cada feature
model.feature_importances_


In [ ]:
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = X.columns,
                                   columns=['importance']).sort_values('importance', ascending=False)
feature_importances

In [ ]:
# Visualizando as importâncias de forma gráfica:
feature_importances.plot(kind='bar')

In [ ]:
prophet_df = df_sem_outlier[['COD_CICLO', 'COD_MATERIAL','Marca_Material','VL_PRECO','FLG_CAMPANHA_MKT_B','FLG_CAMPANHA_MKT_D', 
'PCT_DESCONTO','Categoria_Material','FLG_DATA','QT_VENDA']]

In [ ]:
bst = xgb.XGBClassifier(max_depth=2)


In [ ]:
bst.fit(X_train, y_train)

In [ ]:
y_pred = bst.predict(X_test)
acc = accuracy_score(y_test, y_pred.round())
print(acc, '\n')